# 📊 Reporting Factory Dashboard
Easily browse and validate auto-generated reports.

In [0]:
from pyspark.sql import functions as F
import json

# Metadata path (update if needed)
metadata_path = "abfss://kyc-data@reprotingfactorydl.dfs.core.windows.net/finance/kyc/metadata/report_definitions.json"
metadata_df = spark.read.option("multiline", "true").json(metadata_path)
report_list = metadata_df.toJSON().map(json.loads).collect()

## 📌 Available Reports

In [0]:
display(metadata_df.select("report_name", "format", "certify", "notify"))

## 🔽 Select and Explore a Report

In [0]:
dbutils.widgets.dropdown("report_name", report_list[0]["report_name"], [r["report_name"] for r in report_list])

In [0]:
selected_report = dbutils.widgets.get("report_name")

In [0]:
df = spark.table(f"finance.kyc_ml.{selected_report}")
display(df)

## 🧾 Schema Summary

In [0]:
df.printSchema()

## 📈 Quick Summary Stats

In [0]:
numeric_cols = [f.name for f in df.schema.fields if str(f.dataType) in ['DoubleType', 'IntegerType', 'LongType']]
summary = df.select(numeric_cols).summary("count", "mean", "stddev", "min", "max")
display(summary)

## 🏷️ Unity Catalog Lineage Tags

In [0]:
spark.sql(f"SHOW TBLPROPERTIES {selected_report}").show(truncate=False)